In [1]:
import prosperity3bt
import pandas as pd
import itertools
import numpy as np
from tqdm import tqdm

from prosperity3bt.runner import run_backtest
from prosperity3bt.models import TradeMatchingMode
from prosperity3bt.file_reader import FileSystemReader
from pathlib import Path

from importlib import reload

In [2]:
def day_profit(result):
    last_timestamp = result.activity_logs[-1].timestamp

    profit = 0
    for row in reversed(result.activity_logs):
        if row.timestamp != last_timestamp:
            break

        profit += row.columns[-1]

    return profit

def backtest_days(trader, days=[-2, -1, 0], data_path='.', round_num=1):
    profits = []
    for day in days:
        backtest_results = prosperity3bt.runner.run_backtest(trader, file_reader=FileSystemReader(Path(data_path)), round_num=round_num, day_num=day, print_output=False, trade_matching_mode=TradeMatchingMode.all, no_names=True, show_progress_bar=False)
        profit = day_profit(backtest_results)
        profits.append(profit)
    return profits

def generate_param_combinations(param_grid):
    param_names = param_grid.keys()
    param_values = param_grid.values()
    combinations = list(itertools.product(*param_values))
    return [dict(zip(param_names, combination)) for combination in combinations]

In [6]:
import kin_basket_trader

reload(kin_basket_trader)


PARAMS = {
    "sma_window_size": 125,
    "std_window_size": 30,
    "thresh_z": 8
}

param_grid = {
    'sma_window_size': [100, 200, 300],
    'std_window_size': [25, 60, 80],
    'thresh_z': [4,6,8]
}

param_profits = []

for params in tqdm(generate_param_combinations(param_grid)):
    trader = kin_basket_trader.Trader(params)
    days = [-1, 0, 1]
    profits = backtest_days(trader, days=days, data_path='.', round_num=2)
    for d in days:
        params[f'profit_day_{d}'] = profits[days.index(d)]
    param_profits.append(params)

100%|██████████| 27/27 [04:48<00:00, 10.68s/it]


In [7]:
df = pd.DataFrame(param_profits)
df['tot_profit'] = df[[f'profit_day_{d}' for d in days]].sum(axis=1)
df.sort_values(by='tot_profit', ascending=False, inplace=True)
df

,sma_window_size,std_window_size,thresh_z,profit_day_-1,profit_day_0,profit_day_1,tot_profit
1,100,25,6,-10348.0,4105.5,15884.0,9641.5
13,200,60,6,-578.0,-324.5,2908.0,2005.5
16,200,80,6,-247.0,-713.5,2826.0,1865.5
14,200,60,8,-1057.0,-2172.0,3472.0,243.0
23,300,60,8,-283.0,-503.5,0.0,-786.5
17,200,80,8,-1325.0,-2172.0,855.0,-2642.0
8,100,80,8,0.0,-3259.0,0.0,-3259.0
26,300,80,8,0.0,-3259.0,0.0,-3259.0
5,100,60,8,0.0,-3259.0,0.0,-3259.0
3,100,60,4,-7006.0,3995.5,-1050.0,-4060.5


In [9]:
df[df['profit_day_-1'] > 0]

,sma_window_size,std_window_size,thresh_z,profit_day_-1,profit_day_0,profit_day_1,tot_profit
21,300,60,4,7642.0,-7126.5,-12188.0,-11672.5
24,300,80,4,680.0,-5168.5,-8893.0,-13381.5


In [12]:
profits

[0.0]

In [ ]:
df.to_csv('backtests/inktrader_bt_results.csv')

,ink_change_threshold_pct,ink_window_size,profit_day_-2,profit_day_-1,profit_day_0,tot_profit
0,0.02,5,4273.0,3450.0,0.0,7723.0
1,0.02,10,4121.0,3450.0,0.0,7571.0
2,0.02,15,4121.0,3450.0,0.0,7571.0
3,0.03,5,0.0,3450.0,0.0,3450.0
4,0.03,10,0.0,3450.0,0.0,3450.0
5,0.03,15,0.0,3450.0,0.0,3450.0
6,0.04,5,0.0,3450.0,0.0,3450.0
7,0.04,10,0.0,3450.0,0.0,3450.0
8,0.04,15,0.0,3450.0,0.0,3450.0
